In [1]:
import matplotlib as plt
import pandas as pd
import numpy as np
import bs4
import re
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup

In [2]:
keyWord = 'data science'
keyWord = keyWord.replace(' ', '%20')
urlorig='https://www.headhunter.com/jobs?Keywords='+keyWord+'&Radius=150&PageNumber=1&OrderBy=Relevance'
urlorig

'https://www.headhunter.com/jobs?Keywords=data%20science&Radius=150&PageNumber=1&OrderBy=Relevance'

In [3]:
def get_pages(soup_param, res_list):
    pages = soup_param.findAll("a", class_ = "searchnext")
    if len(pages) > 0:
        link = pages[0]
        link_esc = link["href"].replace(' ', '%20')
        print(link["href"])
        Jb_page = uReq("https://www.headhunter.com" + link_esc)
        Jb_page_html = Jb_page.read()
        Jb_page_soup = soup(Jb_page_html, 'html.parser')
        Jb_page.close()
        res_list.append(Jb_page_soup)
        get_pages(Jb_page_soup, res_list)


my_url = urlorig
uClient = uReq(my_url)
page_html = uClient.read()
uClient.close()
page_soup = soup(page_html,'html.parser')

jb_all = []


links_all_jobs = page_soup.findAll("a", class_ = "see_all_jobs_link")
for link in links_all_jobs:
    Jb_all = uReq("https://www.headhunter.com" + link["href"])
    print(link["href"])
    Jb_all_html = Jb_all.read()
    Jb_all.close()
    Jb_all_soup = soup(Jb_all_html, "html.parser")
    jb_all.append(Jb_all_soup)
    
all_pages=[]
for item in jb_all:
    get_pages(item, all_pages)

jb_all.extend(all_pages)


/jobs?CompanyFacet=%22Robert+Half+Technology%22&CountryCode=US&Keywords=data+science&OrderBy=Relevance&Radius=150
/jobs?CompanyFacet=%22TEKsystems%2C+Inc%22&CountryCode=US&Keywords=data+science&OrderBy=Relevance&Radius=150
/jobs?CompanyFacet=%22Robert+Half+Finance+%26+Accounting+U.S%22&CountryCode=US&Keywords=data+science&OrderBy=Relevance&Radius=150
/jobs?CompanyFacet="Robert Half Technology"&CountryCode=US&Keywords=data science&OrderBy=Relevance&Radius=150&HostSite=AH&UseFacets=true&PageNumber=2
/jobs?CompanyFacet="Robert Half Technology"&CountryCode=US&Keywords=data science&OrderBy=Relevance&Radius=150&HostSite=AH&UseFacets=true&PageNumber=3
/jobs?CompanyFacet="Robert Half Technology"&CountryCode=US&Keywords=data science&OrderBy=Relevance&Radius=150&HostSite=AH&UseFacets=true&PageNumber=4
/jobs?CompanyFacet="Robert Half Technology"&CountryCode=US&Keywords=data science&OrderBy=Relevance&Radius=150&HostSite=AH&UseFacets=true&PageNumber=5


In [ ]:
#single vacancy page
# my_url1 = "https://www.headhunter.com/jobs/Data-Analytics-Manager/J3M2FR6FRKHG9GZFN1W"
my_url1 = "https://www.headhunter.com/jobs/Senior-Data-Scientist/J3N61M605D08SG4PV0C"
uClient1 = uReq(my_url1)
page_html1 = uClient1.read()
uClient1.close()
page_soup1 = soup(page_html1,'html.parser')

print(page_soup1)



In [ ]:
# content = page_soup1.findAll("div", class_="content")
# print(len(content))
# for i in content:
#     print(i.text)
#     refID = re.search("Ref ID: [0-9]+-+[0-9]+", content.text)
#     print(refID.group()[8:])

job_text = page_soup1.findAll("div", id=("description-container-premium", "description-container-standard"))
item = job_text[0]
contents = item.select("div.content")
# print(contents)

refID = re.search("Ref ID: [0-9]+-+[0-9]+", contents[0].text)
if refID != None:
    print(refID.group()[8:])

descr=''
for content in contents:

    
    descr+= content.text
    
    for i in content.select("div#job-snapshot-container.content div.jobs-detail-summary-item span"):
        if i.text == 'Location ':
            print(i.next_sibling.string.next_sibling.text)
        if i.text == 'Employment Type ':
            print(i.next_sibling.string.next_sibling.text)
        if i.text == 'Pay Type ':
            print(i.next_sibling.string.next_sibling.text)
        if i.text == 'Pay Rate ':
            print(i.next_sibling.string.next_sibling.text)
        if i.text == 'Store Type ':
            print(i.next_sibling.string.next_sibling.text)
        if i.text == 'Other Compensation: ':
            print(i.next_sibling.string.next_sibling.text)
# print(descr)

if("statistic" in descr.lower()):
    print('1')
else:
    print('0')

In [4]:
titles = []
company_names = []
location_names = []
compensation = []
job_descriptions = []
job_ids = []
employment_types = []
pay_types=[]
pay_rates=[]
store_types=[]
other_compensations=[]
refs=[]
skills = {}
skills['Python'] = []
skills['R '] = []
skills['Matlab'] = []
skills['Excel'] = []
skills['SAS'] = []
skills['SQL'] = []
skills['SPSS'] = []
skills['Hadoop'] = []
skills['Spark'] = []
skills['Algebra'] = []
skills['Statistic'] = []
skills['Mathematics'] = []
skills['Reinforcement_Learning'] = []
skills['Deep_Learning'] = []
skills['Machine_Learning'] = []
skills['Data_Visualization'] = []
skills['Keras'] = []
skills['TensorFlow'] = []
skills['Artificial_Intelligence'] = []
skills['Communications'] = []
skills['Team_Player'] = []
skills['Empathy'] = []
skills['Emotional_Intellect'] = []
skills['Self_Improvement'] = []
skills['Presentation'] = []
skills['Critical_Thinking'] = []
skills['Business_Understanding'] = []



# extract job_title
jobs = page_soup.findAll("div", class_=re.compile("searchresult clearfix"))
for job in jobs:
    try:
        titles.append(job["title"])
    except:
        titles.append("NA")

# print(titles)

# extract company_name
companies = page_soup.findAll("span", class_="resultcompany_mobile clearfix" )
for company in companies:
    try:
        company_names.append(company.text.strip())
    except:
        company_names.append("NA")
        
# print(company_names)

# extract location       
locations = page_soup.findAll("span", class_="result-right-one")
for location in locations:
    try:
        location_names.append(location.text)
    except:
        location_names.append("NA")
        
# print(location_names)

# extract compensation       
payments = page_soup.findAll("span", class_="result-right-two non-mobile")
for payment in payments:
    try:
        compensation.append(payment.text)
    except:
        compensation.append("NA")
        
# print(compensation)

#extract job_id and full description
links = page_soup.findAll("a", class_ = "result-left")
ind=0
for link in links:
    ind+=1
    print(ind, link["href"])
    Jb = uReq("https://www.headhunter.com" + link["href"])
    Jb_html = Jb.read()
    Jb.close()
    Jb_soup = soup(Jb_html, "html.parser")
    
    try:
        refs.append("https://www.headhunter.com" + link["href"])
    except:
        refs.append("NA")
    
    job_text = Jb_soup.findAll("div", id=("description-container-premium", "description-container-standard"))
    item = job_text[0]
    contents = item.select("div.content")

    refID = re.search("Ref ID: [0-9]+-+[0-9]+", contents[0].text)
    try:
        job_ids.append(refID.group()[8:])
    except:
        job_ids.append("NA")
    
    descr=''
    other_comp_flag=False
    for content in contents:
        
        descr+= content.text
        
    
    
        for i in content.select("div#job-snapshot-container.content div.jobs-detail-summary-item span"):
    #         if i.text == 'Location ':
    #             print(i.next_sibling.string.next_sibling.text)

    #         other_compensations.append("NA")

            if i.text == 'Employment Type ':
                try:
                    employment_types.append(i.next_sibling.string.next_sibling.text)
                except:
                    employment_types.append("NA")
            if i.text == 'Pay Type ':
                try:
                    pay_types.append(i.next_sibling.string.next_sibling.text)
                except:
                    pay_types.append("NA")
            if i.text == 'Pay Rate ':
                try:
                    pay_rates.append(i.next_sibling.string.next_sibling.text)
                except:
                    pay_rates.append("NA")
            if i.text == 'Store Type ':
                try:
                    store_types.append(i.next_sibling.string.next_sibling.text)
                except:
                    store_types.append("NA")
            if i.text == 'Other Compensation: ':
                other_comp_flag=True
                try:
                    other_compensations.append(i.next_sibling.string.next_sibling.text)
                except:
                    other_compensations.append("NA")
                
    try:
        job_descriptions.append(str(re.sub('(\n|\r)', '',descr)))
    except:
        job_descriptions.append("NA")
        
# looking for skills - Excell
    if("excel" in descr.lower()):
        skills['Excel'].append('1')
    else:
        skills['Excel'].append('0')
# looking for skills - Python    
    if("python" in descr.lower()):
        skills['Python'].append('1')
    else:
        skills['Python'].append('0')
# looking for skills - R
    if("R " in descr):
        skills['R '].append('1')
    else:
        skills['R '].append('0')
# looking for skills - Matlab
    if("matlab" in descr.lower()):
        skills['Matlab'].append('1')
    else:
        skills['Matlab'].append('0')
# looking for skills - SAS
    if("sas" in descr.lower()):
        skills['SAS'].append('1')
    else:
        skills['SAS'].append('0')
# looking for skills - SQL
    if("sql" in descr.lower()):
        skills['SQL'].append('1')
    else:
        skills['SQL'].append('0')
# looking for skills - SPSS
    if("spss" in descr.lower()):
        skills['SPSS'].append('1')
    else:
        skills['SPSS'].append('0')
# looking for skills - Hadoop
    if("hadoop" in descr.lower()):
        skills['Hadoop'].append('1')
    else:
        skills['Hadoop'].append('0')
# looking for skills - Spark
    if("spark" in descr.lower()):
        skills['Spark'].append('1')
    else:
        skills['Spark'].append('0')
# looking for skills - Algebra
    if("algebra" in descr.lower()):
        skills['Algebra'].append('1')
    else:
        skills['Algebra'].append('0')
# looking for skills - Statistic
    if("statistic" in descr.lower()):
        skills['Statistic'].append('1')
    else:
        skills['Statistic'].append('0')
# looking for skills - Mathematics
    if("math" in descr.lower()):
        skills['Mathematics'].append('1')
    else:
        skills['Mathematics'].append('0')
# looking for skills - Reinforcement Learning
    if("reinforcement learning" in descr.lower()):
        skills['Reinforcement_Learning'].append('1')
    else:
        skills['Reinforcement_Learning'].append('0')
# looking for skills - Deep Learning    
    if("deep learning" in descr.lower()):
        skills['Deep_Learning'].append('1')
    else:
        skills['Deep_Learning'].append('0')
# looking for skills - Machine Learning                                   
    if("machine learning" in descr.lower()):
        skills['Machine_Learning'].append('1')
    else:
        skills['Machine_Learning'].append('0')
# looking for skills - Data Visualization
    if("visualization" in descr.lower()):
        skills['Data_Visualization'].append('1')
    else:
        skills['Data_Visualization'].append('0')
# looking for skills - Keras
    if("keras" in descr.lower()):
        skills['Keras'].append('1')
    else:
        skills['Keras'].append('0')
# looking for skills - TensorFlow
    if("tensorflow" in descr.lower()):
        skills['TensorFlow'].append('1')
    else:
        skills['TensorFlow'].append('0')
# looking for skills - Artificial Intelligence
    if("artificial intelligence" in descr.lower()):
        skills['Artificial_Intelligence'].append('1')
    else:
        skills['Artificial_Intelligence'].append('0')
# looking for skills - Communications
    if("communication" in descr.lower()):
        skills['Communications'].append('1')
    else:
        skills['Communications'].append('0')
# looking for skills - Team Player
    if("team work" in descr.lower() or "team spirit" in descr.lower() or "team member" in descr.lower() 
       or "team player" in descr.lower()):
        skills['Team_Player'].append('1')
    else:
        skills['Team_Player'].append('0')
# looking for skills - Empathy
    if("empathy" in descr.lower()):
        skills['Empathy'].append('1')
    else:
        skills['Empathy'].append('0')
# looking for skills - Emotional Intellect
    if("emotional intellect" in descr.lower()):
        skills['Emotional_Intellect'].append('1')
    else:
        skills['Emotional_Intellect'].append('0')
# looking for skills - Self-Improvement
    if("self-improvement" in descr.lower() or "personal development" in descr.lower()):
        skills['Self_Improvement'].append('1')
    else:
        skills['Self_Improvement'].append('0')
# looking for skills - Presentation
    if("presentation" in descr.lower()):
        skills['Presentation'].append('1')
    else:
        skills['Presentation'].append('0')
# looking for skills - Critical Thinking
    if("critical thinking" in descr.lower()):
        skills['Critical_Thinking'].append('1')
    else:
        skills['Critical_Thinking'].append('0')
# looking for skills - Business Understanding
    if("business understanding" in descr.lower() or "understand business" in descr.lower()):
        skills['Business_Understanding'].append('1')
    else:
        skills['Business_Understanding'].append('0')
    
    if other_comp_flag==False:
        other_compensations.append("NA")
    
#     try:
#         content = Jb_soup.find("div", class_="content")
#         refID = re.search("Ref ID: [0-9]+-+[0-9]+", content.text)
#         job_ids.append(refID.group()[8:])
#     except:
#         job_ids.append("NA")    
        
#     try:
#         job_descriptions.append(str(re.sub('(\n|\r)', '',content.text)))
#     except:
#         job_descriptions.append("NA")
        
        
# print(job_descriptions)

#extract all jobsfor company with many vacancies
for item in jb_all:
    
    # extract job_title
    jobs = item.findAll("div", class_=re.compile("searchresult clearfix"))
    for job in jobs:
        try:
            titles.append(job["title"])
        except:
            titles.append("NA")
            
    # extract company_name
    companies = item.findAll("span", class_="resultcompany_mobile clearfix" )
    for company in companies:
        try:
            company_names.append(company.text.strip())
        except:
            company_names.append("NA")

    # extract location       
    locations = item.findAll("span", class_="result-right-one")
    for location in locations:
        try:
            location_names.append(location.text)
        except:
            location_names.append("NA")

    # extract compensation       
    payments = item.findAll("span", class_="result-right-two non-mobile")
    for payment in payments:
        try:
            compensation.append(payment.text)
        except:
            compensation.append("NA")

    #extract job description and all information rom it
    links = item.findAll("a", class_ = "result-left")
    for link in links:
        ind+=1
        print(ind, link["href"])
        Jb = uReq("https://www.headhunter.com" + link["href"])
        Jb_html = Jb.read()
        Jb.close()
        Jb_soup = soup(Jb_html, "html.parser")
        
        try:
            refs.append("https://www.headhunter.com" + link["href"])
        except:
            refs.append("NA")

        job_text = Jb_soup.findAll("div", id=("description-container-premium", "description-container-standard"))
        item = job_text[0]
        contents = item.select("div.content")

        refID = re.search("Ref ID: [0-9]+-+[0-9]+", contents[0].text)
        try:
            job_ids.append(refID.group()[8:])
        except:
            job_ids.append("NA")

        descr=''
        other_comp_flag=False
        for content in contents:
        
            descr+= content.text

            for i in content.select("div#job-snapshot-container.content div.jobs-detail-summary-item span"):
        #         if i.text == 'Location ':
        #             print(i.next_sibling.string.next_sibling.text)

        #         other_compensations.append("NA")

                if i.text == 'Employment Type ':
                    try:
                        employment_types.append(i.next_sibling.string.next_sibling.text)
                    except:
                        employment_types.append("NA")
                if i.text == 'Pay Type ':
                    try:
                        pay_types.append(i.next_sibling.string.next_sibling.text)
                    except:
                        pay_types.append("NA")
                if i.text == 'Pay Rate ':
                    try:
                        pay_rates.append(i.next_sibling.string.next_sibling.text)
                    except:
                        pay_rates.append("NA")
                if i.text == 'Store Type ':
                    try:
                        store_types.append(i.next_sibling.string.next_sibling.text)
                    except:
                        store_types.append("NA")
                if i.text == 'Other Compensation: ':
                    other_comp_flag=True
                    try:
                        other_compensations.append(i.next_sibling.string.next_sibling.text)
                    except:
                        other_compensations.append("NA")

        try:
            job_descriptions.append(str(re.sub('(\n|\r)', '',descr)))
        except:
            job_descriptions.append("NA")
            
    # looking for skills - Excell
        if("excel" in descr.lower()):
            skills['Excel'].append('1')
        else:
            skills['Excel'].append('0')
    # looking for skills - Python    
        if("python" in descr.lower()):
            skills['Python'].append('1')
        else:
            skills['Python'].append('0')
    # looking for skills - R
        if("R " in descr):
            skills['R '].append('1')
        else:
            skills['R '].append('0')
    # looking for skills - Matlab
        if("matlab" in descr.lower()):
            skills['Matlab'].append('1')
        else:
            skills['Matlab'].append('0')
    # looking for skills - SAS
        if("sas" in descr.lower()):
            skills['SAS'].append('1')
        else:
            skills['SAS'].append('0')
    # looking for skills - SQL
        if("sql" in descr.lower()):
            skills['SQL'].append('1')
        else:
            skills['SQL'].append('0')
    # looking for skills - SPSS
        if("spss" in descr.lower()):
            skills['SPSS'].append('1')
        else:
            skills['SPSS'].append('0')
    # looking for skills - Hadoop
        if("hadoop" in descr.lower()):
            skills['Hadoop'].append('1')
        else:
            skills['Hadoop'].append('0')
    # looking for skills - Spark
        if("spark" in descr.lower()):
            skills['Spark'].append('1')
        else:
            skills['Spark'].append('0')
    # looking for skills - Algebra
        if("algebra" in descr.lower()):
            skills['Algebra'].append('1')
        else:
            skills['Algebra'].append('0')
    # looking for skills - Statistic
        if("statistic" in descr.lower()):
            skills['Statistic'].append('1')
        else:
            skills['Statistic'].append('0')
    # looking for skills - Mathematics
        if("math" in descr.lower()):
            skills['Mathematics'].append('1')
        else:
            skills['Mathematics'].append('0')
    # looking for skills - Reinforcement Learning
        if("reinforcement learning" in descr.lower()):
            skills['Reinforcement_Learning'].append('1')
        else:
            skills['Reinforcement_Learning'].append('0')
    # looking for skills - Deep Learning    
        if("deep learning" in descr.lower()):
            skills['Deep_Learning'].append('1')
        else:
            skills['Deep_Learning'].append('0')
    # looking for skills - Machine Learning                                   
        if("machine learning" in descr.lower()):
            skills['Machine_Learning'].append('1')
        else:
            skills['Machine_Learning'].append('0')
    # looking for skills - Data Visualization
        if("visualization" in descr.lower()):
            skills['Data_Visualization'].append('1')
        else:
            skills['Data_Visualization'].append('0')
    # looking for skills - Keras
        if("keras" in descr.lower()):
            skills['Keras'].append('1')
        else:
            skills['Keras'].append('0')
    # looking for skills - TensorFlow
        if("tensorflow" in descr.lower()):
            skills['TensorFlow'].append('1')
        else:
            skills['TensorFlow'].append('0')
    # looking for skills - Artificial Intelligence
        if("artificial intelligence" in descr.lower()):
            skills['Artificial_Intelligence'].append('1')
        else:
            skills['Artificial_Intelligence'].append('0')
    # looking for skills - Communications
        if("communication" in descr.lower()):
            skills['Communications'].append('1')
        else:
            skills['Communications'].append('0')
    # looking for skills - Team Player
        if("team work" in descr.lower() or "team spirit" in descr.lower() or "team member" in descr.lower() 
           or "team player" in descr.lower()):
            skills['Team_Player'].append('1')
        else:
            skills['Team_Player'].append('0')
    # looking for skills - Empathy
        if("empathy" in descr.lower()):
            skills['Empathy'].append('1')
        else:
            skills['Empathy'].append('0')
    # looking for skills - Emotional Intellect
        if("emotional intellect" in descr.lower()):
            skills['Emotional_Intellect'].append('1')
        else:
            skills['Emotional_Intellect'].append('0')
    # looking for skills - Self-Improvement
        if("self-improvement" in descr.lower() or "personal development" in descr.lower()):
            skills['Self_Improvement'].append('1')
        else:
            skills['Self_Improvement'].append('0')
    # looking for skills - Presentation
        if("presentation" in descr.lower()):
            skills['Presentation'].append('1')
        else:
            skills['Presentation'].append('0')
    # looking for skills - Critical Thinking
        if("critical thinking" in descr.lower()):
            skills['Critical_Thinking'].append('1')
        else:
            skills['Critical_Thinking'].append('0')
    # looking for skills - Business Understanding
        if("business understanding" in descr.lower() or "understand business" in descr.lower()):
            skills['Business_Understanding'].append('1')
        else:
            skills['Business_Understanding'].append('0')

        if other_comp_flag==False:
            other_compensations.append("NA")

1 /jobs/Senior-Data-Scientist/J3N61M605D08SG4PV0C
2 /jobs/Sr--Data-Engineer/J3Q1YB68FNWG2M32XTG
3 /jobs/Senior-Data-Scientist/J3M0HJ6G44D9GHZ6KSW
4 /jobs/Manager-of-IT-Audit-and-Data-Analytics/J3S11Z6T9RXPD3MPF70
5 /jobs/Data-Analytics-Manager/J3M2FR6FRKHG9GZFN1W
6 /jobs/Data-Manager/J3P8B46ZD1B072GBXK6
7 /jobs/Sr--Data-Engineer/J3W2QX77VC8F80WYZR4
8 /jobs/Senior-Data-Engineer/J3N2BM77MJY8XMZ8VH6
9 /jobs/Director--Analytics/J3R3L86W1CQ32H1M8FM
10 /jobs/Tax-Analyst-Manager-Director/J3W2GX73DG4MDBLBPJD
11 /jobs/Sr--Financial-Analyst-Excellent-Culture-Team--Luv-/J3V4Z869LYN28RG7T9B
12 /jobs/Sr--Financial-Analyst-Great-Work-Life-Balance--Luv-/J3R46G64SVM36Y6YV0H
13 /jobs/Senior-Manager--Customer-Analytics--Spoke-Role/J3P7DS6Q6WG8N6K17MD
14 /jobs/Clinical-Project-Manager/J3N3126BLBKJLS89B7C
15 /jobs/Quality-Control-Manager/J3N5XQ6GVFDSTP845L2
16 /jobs/Senior-Clinical-Trial-Manager/J3V89K78457PBXV81KV
17 /jobs/Sr--Business-Analyst/J3P73X63SZHYHBYLP2L
18 /jobs/Modernization-Quality-Assurance-

146 /jobs/Sr--Business-Intelligence-Analyst/J3T4QN6K4B51FDTQPHR
147 /jobs/Director-of-IT-Audit/J3T5WF77PVB4KZ3C1FP
148 /jobs/Sr--Network-Security-Analyst/J3R3SH64TJ5T21JHYXN
149 /jobs/Sr-Front-End-Web-Developer--Angular--Unity--C---AWS--MySQL-/J3N1W26RLQ7XWQDCR8W
150 /jobs/Sr--C--PHP-Engineer/J3W59S78ZPH4VH8TKT4
151 /jobs/Senior-Consultant--Application-Security/J3Q85K6LV0MPSM79FNJ
152 /jobs/Sr--C--PHP-Engineer/J3N61S60ZCPDT9LN4MF
153 /jobs/Sr--Software-Engineer/J3N2ZP6MX12F3H84529
154 /jobs/Sr--C--PHP-Engineer/J3S2QF7082BTC87VD9R
155 /jobs/Sr--C--PHP-Engineer/J3V3606TK881W1J0066
156 /jobs/Sr--Software-Engineer/J3Q7XQ6X2FJBFVZRKNH
157 /jobs/Sr--Infrastructure-Manager-Systems-Admin/J3T8F56YLM9W0KMT2DY
158 /jobs/Senior-Systems-Administrator/J3V2ZV6R156DYKZV106
159 /jobs/Senior-Network-Administrator--Engineer-/J3R20N6TTXBWQBLGG12
160 /jobs/IT-Manager/J3V3R76P6PSP8HPS6DZ
161 /jobs/Sr--PHP-Developer/J3R7H46YSS8TPK6JMCX
162 /jobs/NOC-Manager/J3R7DH74WX64G4LRQ9X
163 /jobs/Sr--Engineer-Industri

In [5]:
print(len(refs))

169


In [6]:
d = {'Job_Title':titles,'Company_Name':company_names, 'Location':location_names, 'Compensation': compensation, 
     'Job_ID':job_ids,'Job_Description':job_descriptions, 'Employment_Type':employment_types, 'Pay_Type':pay_types,
     'Pay_Rate':pay_rates, 'Store_Type':store_types, 'Other_Compensation':other_compensations, 'Link': refs}

df1 = pd.DataFrame(d)
df2 = pd.DataFrame(skills)
frames = [df1, df2]
results = pd.concat(frames, axis = 1)
results.head()

,Job_Title,Company_Name,Location,Compensation,Job_ID,Job_Description,Employment_Type,Pay_Type,Pay_Rate,Store_Type,...,TensorFlow,Artificial_Intelligence,Communications,Team_Player,Empathy,Emotional_Intellect,Self_Improvement,Presentation,Critical_Thinking,Business_Understanding
0,Senior-Data-Scientist,Robert Half Technology,TX - Fort Worth,N/A,04030-0011057821,Ref ID: 04030-0011057821Classification: Data S...,Full-Time,Year,N/A,"Engineering, Information Technology, Design",...,0,0,1,1,0,0,0,0,0,0
1,Sr--Data-Engineer,Robert Half Technology,NJ - Princeton,N/A,02720-0011260365,Ref ID: 02720-0011260365Classification: Sr. So...,Full-Time,Year,N/A,"Engineering, Information Technology, Design",...,0,0,0,0,0,0,0,0,0,0
2,Senior-Data-Scientist,Robert Half Technology,NJ - Jersey City,$140k - $160k/year,02710-0011220068,Ref ID: 02710-0011220068Classification: Data S...,Full-Time,Year,"$140,000.00 - $160,000.00 /Year","Engineering, Information Technology, Design",...,0,1,0,0,0,0,0,0,0,0
3,Manager-of-IT-Audit-and-Data-Analytics,Robert Half Technology,GA - Atlanta,$120k - $130k/year,00900-0011225244,Ref ID: 00900-0011225244Classification: Sr. IT...,Full-Time,Year,"$120,000.00 - $130,000.00 /Year","Engineering, Information Technology, Design",...,0,0,0,1,0,0,0,0,0,0
4,Data-Analytics-Manager,"TEKsystems, Inc",GA - Atlanta,N/A,NA,***** CANDIDATES MUST BE ABLE TO WORK W2 ****...,Full-Time,Year,N/A,Information Technology,...,0,0,0,0,0,0,0,0,0,0


In [7]:
results.tail()

,Job_Title,Company_Name,Location,Compensation,Job_ID,Job_Description,Employment_Type,Pay_Type,Pay_Rate,Store_Type,...,TensorFlow,Artificial_Intelligence,Communications,Team_Player,Empathy,Emotional_Intellect,Self_Improvement,Presentation,Critical_Thinking,Business_Understanding
164,Senior-IT-Analyst---Network-Administrator,Robert Half Technology,IA - Davenport,$90k - $95k/year,01520-0011195026,Ref ID: 01520-0011195026Classification: Networ...,Full-Time,Year,"$90,000.00 - $95,000.00 /Year","Engineering, Information Technology, Design",...,0,0,0,0,0,0,0,0,0,0
165,Senior-Enterprise-Security-Risk-Analyst,Robert Half Technology,FL - Jacksonville,N/A,01120-0011223974,Ref ID: 01120-0011223974Classification: Data S...,Full-Time,Year,N/A,"Engineering, Information Technology, Design",...,0,0,0,0,0,0,0,0,0,0
166,Senior-Systems-Administrator,Robert Half Technology,IL - Moline,$70k - $100k/year,01520-0011228212,Ref ID: 01520-0011228212Classification: System...,Full-Time,Year,"$70,000.00 - $100,000.00 /Year","Engineering, Information Technology, Design",...,0,0,0,1,0,0,0,0,0,0
167,Software-Configuration-Manager,Robert Half Technology,IA - Iowa City,N/A,01520-0011209078,Ref ID: 01520-0011209078Classification: Softwa...,Full-Time,Year,N/A,"Engineering, Information Technology, Design",...,0,0,0,0,0,0,0,0,0,0
168,Sr--Business-Intelligence-Analyst,Robert Half Technology,NJ - Bridgewater Township,N/A,02720-0011196585,Ref ID: 02720-0011196585Classification: Busine...,Full-Time,Year,N/A,"Engineering, Information Technology, Design",...,0,0,1,0,0,0,0,0,0,0


In [8]:
print(len(results))
result_unique = results.drop_duplicates()
print(len(result_unique))

169
157


In [9]:
result_unique.to_csv('Headhunters_job_vacancies.csv')